In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [ ]:
data = pd.read_csv('data/Airbnb_Texas_Rentals.csv', index_col=0)

<b> Data characteristics </b>

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
print('Unique values count for each column: ')
{i:data[i].nunique() for i in data}

In [ ]:
print('Duplicated values count for each column: ')

{i:data[i].duplicated().sum() for i in data}

In [ ]:
print('Nan values count for each column: ')

{i:data[i].isna().sum() for i in data}

<b> Columns preprocessing </b>

In [ ]:
data['average_rate_per_night'][~(data['average_rate_per_night'].str.startswith('$').astype('bool').values)]

In [ ]:
data['average_rate_per_night_dollar'] = data['average_rate_per_night'].apply(lambda row: None if pd.isna(row) else row[1:]).values
data['average_rate_per_night_dollar'] = data['average_rate_per_night_dollar'].astype('float64')

In [ ]:
(data.bedrooms_count=='0').sum()

In [ ]:
data.bedrooms_count[~(data.bedrooms_count.str.contains('\\d', regex=True).astype('bool'))]

In [ ]:
data['bedrooms_count_int'] = data['bedrooms_count'].replace('Studio', '0').astype('float64')

In [ ]:
data['city_chng'] = data['city'].str.lower().str.strip().values

In [ ]:
data['date_of_listing'].values

In [ ]:
data['month'] = data['date_of_listing'].apply(lambda row: row.split(' ')[0]).astype(str).values
data['year'] = data['date_of_listing'].apply(lambda row: row.split(' ')[1]).astype(int).values

In [ ]:
data['latitude'] = data['latitude'].astype('float64')
data['longitude'] = data['longitude'].astype('float64')

<b> Data EDA plots</b>

In [ ]:
data.average_rate_per_night_dollar.describe()

In [ ]:
import numpy as np
quants = np.quantile(data.average_rate_per_night_dollar.fillna(0), np.linspace(0, 0.9, 9))

In [ ]:
quants

In [ ]:
data.average_rate_per_night_dollar.hist(bins=quants)
plt.xticks(quants)
plt.title('Average rate per night distribution')
plt.xlabel('Avg Rate per night')
plt.ylabel('Count')
plt.show()

In [ ]:
data.bedrooms_count_int.unique()

In [ ]:
from collections import Counter
cnts = Counter(data.bedrooms_count_int)
plt.bar(cnts.keys(), cnts.values())
plt.xticks(data.bedrooms_count_int.fillna(0).unique())
plt.xlabel('Bedrooms count')
plt.ylabel('Count')
plt.title('Bedrooms count distribution')
plt.show()

In [ ]:
from collections import Counter
cnts = Counter(data.month)
plt.bar(cnts.keys(), cnts.values())
plt.xticks(list(cnts.keys()), rotation=50)
plt.xlabel('Months')
plt.ylabel('Count')
plt.title('Months count distribution')
plt.show()

In [ ]:
from collections import Counter
cnts = Counter(data.year)
plt.bar(cnts.keys(), cnts.values())
plt.xticks(list(cnts.keys()), rotation=50)
plt.xlabel('Years')
plt.ylabel('Count')
plt.title('Years count distribution')
plt.show()

#### Data visualization dashboard

In [ ]:
# app = dash.Dash(__name__)

In [ ]:
# app.layout = html.Div([
#     html.Nav([
#         html.Ul([
#             html.Li(html.A("Overview", href="#page-1")),
#             html.Li(html.A("Detailed Visualizations", href="#page-2")),
#             html.Li(html.A("Cities Comparison", href="#page-3")),
#         ])
#     ]),

#     html.Div([
#         html.H1("Airbnb Data Overview", id="page-1"),
#         html.Div([
#             html.Div([
#                 html.H2("Average Rate per Night"),
#                 dcc.Graph(id='avg_rate_per_night'),
#             ], className="six columns"),
#             html.Div([
#                 html.H2("Bedrooms Count"),
#                 dcc.Graph(id='bedrooms_count'),
#             ], className="six columns"),
#         ], className="row"),
#     ], id='page-1-container'),

#     html.Div([
#         html.H1("Detailed Visualizations", id="page-2"),
#         dcc.Dropdown(
#             id='city-dropdown',
#             options=[{'label': city, 'value': city} for city in data['city'].unique()],
#             value=data['city'].unique()[0]
#         ),
#         dcc.Graph(id='map-plot'),
#         html.Div(id='selected-property')
#     ], id='page-2-container'),

#     html.Div([
#         html.H1("City Comparison", id="page-3"),
#         dcc.Dropdown(
#             id='city1-dropdown',
#             options=[{'label': city, 'value': city} for city in data['city'].unique()],
#             value=data['city'].unique()[0]
#         ),
#         dcc.Dropdown(
#             id='city2-dropdown',
#             options=[{'label': city, 'value': city} for city in data['city'].unique()],
#             value=data['city'].unique()[1]
#         ),
#         dcc.Graph(id='city-comparison-plot'),
#     ], id='page-3-container')
# ])


In [ ]:
# @app.callback(
#     [Output('avg_rate_per_night', 'figure'),
#      Output('bedrooms_count', 'figure')],
#     [Input('city-dropdown', 'value')]
# )
# def update_overview(city):
#     filtered_data = data[data['city'] == city]
#     avg_rate_fig = px.histogram(filtered_data, x='average_rate_per_night_dollar', title='Average Rate per Night Distribution')
#     bedrooms_count_fig = px.bar(filtered_data, x='bedrooms_count', title='Bedrooms Count Distribution')
#     return avg_rate_fig, bedrooms_count_fig

In [ ]:
# @app.callback(
#     [Output('map-plot', 'figure'),
#      Output('selected-property', 'children')],
#     [Input('city-dropdown', 'value')]
# )
# def update_map(city):
#     filtered_data = data[data['city'] == city]
#     map_fig = px.scatter_mapbox(filtered_data, lat='latitude', lon='longitude', hover_name='title', zoom=10)
#     map_fig.update_layout(mapbox_style="open-street-map")

#     selected_property_info = filtered_data.iloc[0] 
#     selected_property_html = html.Div([
#         html.H3("Selected Property Information"),
#         html.P(f"Title: {selected_property_info['title']}"),
#         html.P(f"Description: {selected_property_info['description']}"),
#         html.P(f"URL: {selected_property_info['url']}")
#     ])

#     return map_fig, selected_property_html

In [ ]:
# @app.callback(
#     Output('city-comparison-plot', 'figure'),
#     [Input('city1-dropdown', 'value'),
#      Input('city2-dropdown', 'value')]
# )
# def update_city_comparison(city1, city2):
#     filtered_data1 = data[data['city'] == city1]
#     filtered_data2 = data[data['city'] == city2]

#     city_comparison_fig = px.bar(
#         filtered_data1.merge(filtered_data2, how='outer', on='bedrooms_count'),
#         x='bedrooms_count',
#         y=['average_rate_per_night_dollar_x', 'average_rate_per_night_dollar_y'],
#         title=f'Cities Comparison: {city1} vs {city2}',
#         labels={'average_rate_per_night_dollar_x': city1, 'average_rate_per_night_dollar_y': city2}
#     )
#     return city_comparison_fig

In [ ]:
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go


# @app.callback(
#     Output('city-comparison-plot', 'figure'),
#     [Input('city1-dropdown', 'value'),
#      Input('city2-dropdown', 'value')]
# )
# def update_city_comparison(city1, city2):
#     filtered_data1 = data[data['city'] == city1].sort_values('bedrooms_count')
#     filtered_data2 = data[data['city'] == city2].sort_values('bedrooms_count')

#     fig = make_subplots(rows=1, cols=2, subplot_titles=(city1, city2))

#     fig.add_trace(go.Bar(
#         x=filtered_data1['bedrooms_count'],
#         y=filtered_data1['average_rate_per_night_dollar'],
#         name=city1,
#         marker=dict(color='rgb(158,202,225)'),
#     ), row=1, col=1)

#     fig.add_trace(go.Bar(
#         x=filtered_data2['bedrooms_count'],
#         y=filtered_data2['average_rate_per_night_dollar'],
#         name=city2,
#         marker=dict(color='rgb(50, 171, 96)'),
#     ), row=1, col=2)

#     fig.update_layout(title_text=f'Cities Comparison: {city1} vs {city2}',
#                       xaxis_title='Bedrooms Count',
#                       yaxis_title='Average Rate per Night (Dollar)')

#     return fig


In [ ]:
# if __name__ == '__main__':
#     app.run_server(debug=True)